In [18]:
!git clone https://github.com/nianlonggu/MemSum.git

Cloning into 'MemSum'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 275 (delta 7), reused 2 (delta 2), pack-reused 266
Receiving objects: 100% (275/275), 82.32 MiB | 22.01 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [19]:
import os
os.chdir("MemSum")
!ls

config
data
Data_processing_training_and_testing_for_MemSum.ipynb
human_eval_results
images
MemSum_Human_Evaluation.ipynb
oracle_score_compute.ipynb
README.md
requirements.txt
slides_for_paper.pdf
src
summarizers.py


In [20]:
!pip install -r requirements.txt -q

In [21]:
import torch
torch.__version__

'2.0.0+cu118'

In [22]:
import json
# train_corpus = [ json.loads(line) for line in open("data/custom_data/train_CUSTOM_raw.jsonl") ]
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/dataset/corpusfinal.csv')
train_corpus = []
for i in range(len(df)):
  dick = {}
  dick['text'] = df['document'][i]
  dick['summary'] = df['ext_summary'][i]
  train_corpus.append(dick)

## as an example, we have 100 instances for training
print(len(train_corpus))
print(train_corpus[0].keys())
print(train_corpus[0]["text"][:3])
print(train_corpus[0]["summary"][:3])

1977
dict_keys(['text', 'summary'])
The
The


In [23]:
from src.data_preprocessing.MemSum.utils import greedy_extract
import json
from tqdm import tqdm

In [24]:
# train_corpus = [ json.loads(line) for line in open("data/custom_data/train_CUSTOM_raw.jsonl") ]
df = pd.read_csv('/content/drive/MyDrive/dataset/corpusfinal.csv')
df_train = df.iloc[:int(len(df)*0.7)]
train_corpus = []
for i in range(len(df_train)):
  dick = {}
  dick['text'] = df_train['document'][i].split(".")
  dick['summary'] = df_train['ext_summary'][i].split(".")
  train_corpus.append(dick)

# train_corpus[0]
for data in tqdm(train_corpus):
    high_rouge_episodes = greedy_extract( data["text"], data["summary"], beamsearch_size = 3 )
    indices_list = []
    score_list  = []

    for indices, score in high_rouge_episodes:
        indices_list.append( indices )
        score_list.append(score)

    data["indices"] = indices_list
    data["score"] = score_list

100%|██████████| 1383/1383 [02:53<00:00,  7.97it/s]


In [25]:
with open("data/train_CUSTOM_labelled.jsonl","w") as f:
    for data in train_corpus:
        f.write(json.dumps(data) + "\n")

In [26]:
df_test = df.iloc[int(len(df)*0.7):]
test_corpus = []
df_test
for i,row in df_test.iterrows():
  dick1 = {}
  dick1['text'] = row['document'].split(".")
  dick1['summary'] = str(row ['ext_summary']).split(".")
  test_corpus.append(dick)
with open("data/val_CUSTOM_raw.jsonl","w") as f:
    for data in test_corpus:
        f.write(json.dumps(data) + "\n")

In [27]:
!pip install gdown -q
try:
    os.system("rm -r model")
    os.makedirs("model/")
except:
    pass
!cd model/; gdown --folder https://drive.google.com/drive/folders/1lrwYrrM3h0-9fwWCOmpRkydvmF6hmvmW


if not os.path.exists("model/glove"):
    try:
        os.makedirs("model/glove")
        os.system("mv model/*.pkl model/glove/")
    except:
        pass

Retrieving folder list
Processing file 1SVTHcgWJDvoVCsLfdvkaw5ICkihjUoaH unigram_embeddings_200dim.pkl
Processing file 1SuF4HSe0-IBKWGtc1xqlzMHNDneiLi4- vocabulary_200dim.pkl
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1SVTHcgWJDvoVCsLfdvkaw5ICkihjUoaH
To: /content/MemSum/MemSum/MemSum/model/glove/unigram_embeddings_200dim.pkl
100% 320M/320M [00:01<00:00, 163MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SuF4HSe0-IBKWGtc1xqlzMHNDneiLi4-
To: /content/MemSum/MemSum/MemSum/model/glove/vocabulary_200dim.pkl
100% 4.16M/4.16M [00:00<00:00, 237MB/s]
Download completed


In [28]:
!cd src/MemSum_Full; python train.py -training_corpus_file_name ../../data/train_CUSTOM_labelled.jsonl -validation_corpus_file_name ../../data/val_CUSTOM_raw.jsonl -model_folder ../../model/MemSum_Full/custom_data/200dim/run1/ -log_folder ../../log/MemSum_Full/custom_data/200dim/run1/ -vocabulary_file_name ../../model/glove/vocabulary_200dim.pkl -pretrained_unigram_embeddings_file_name ../../model/glove/unigram_embeddings_200dim.pkl -max_seq_len 100 -max_doc_len 500 -num_of_epochs 5 -save_every 1000 -n_device 1 -batch_size_per_device 2 -max_extracted_sentences_per_document 7 -moving_average_decay 0.999 -p_stop_thres 0.6

1383it [00:00, 67361.75it/s]
594it [00:00, 52776.42it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
0it [00:00, ?it/s]/content/MemSum/MemSum/MemSum/src/MemSum_Full/train.py:227: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  remaining_mask_np =

In [29]:
from summarizers import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np

In [30]:
rouge_cal = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeLsum'], use_stemmer=True)

memsum_custom_data = MemSum( "model/MemSum_Full/custom_data/200dim/run1/model_batch_3455.pt", 
                  "model/glove/vocabulary_200dim.pkl", 
                  gpu = 0 ,  max_doc_len = 500  )

In [32]:
# test_corpus_custom_data = [ json.loads(line) for line in open("data/custom_data/test_CUSTOM_raw.jsonl")]
df_val = pd.read_csv('/content/drive/MyDrive/dataset/corpusfinal.csv')
df_test = df.iloc[:int(len(df)*0.3)]
test_corpus_custom_data = []
df_test
for i,row in df_test.iterrows():
  dick1 = {}
  dick1['text'] = row['document'].split(".")
  dick1['summary'] = str(row['ext_summary']).split(".")
  test_corpus_custom_data.append(dick)

In [33]:
def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        gold_summary = data["summary"]
        extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, max_extracted_sentences_per_document = max_extracted_sentences )[0]
        
        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )
    
    return np.asarray(scores).mean(axis = 0)

In [34]:
evaluate( memsum_custom_data, test_corpus_custom_data, 0.6, 7, rouge_cal )

100%|██████████| 593/593 [01:34<00:00,  6.26it/s]


array([0.73410405, 0.62209302, 0.71676301])

In [35]:
!zip -r ../memsum.zip /content/MemSum/

  adding: content/MemSum/ (stored 0%)
  adding: content/MemSum/human_eval_results/ (stored 0%)
  adding: content/MemSum/human_eval_results/records_memsum_wo_autostop_neusum.jsonl (deflated 76%)
  adding: content/MemSum/human_eval_results/records_memsum_neusum.jsonl (deflated 76%)
  adding: content/MemSum/.git/ (stored 0%)
  adding: content/MemSum/.git/branches/ (stored 0%)
  adding: content/MemSum/.git/logs/ (stored 0%)
  adding: content/MemSum/.git/logs/refs/ (stored 0%)
  adding: content/MemSum/.git/logs/refs/heads/ (stored 0%)
  adding: content/MemSum/.git/logs/refs/heads/main (deflated 27%)
  adding: content/MemSum/.git/logs/refs/remotes/ (stored 0%)
  adding: content/MemSum/.git/logs/refs/remotes/origin/ (stored 0%)
  adding: content/MemSum/.git/logs/refs/remotes/origin/HEAD (deflated 27%)
  adding: content/MemSum/.git/logs/HEAD (deflated 27%)
  adding: content/MemSum/.git/info/ (stored 0%)
  adding: content/MemSum/.git/info/exclude (deflated 28%)
  adding: content/MemSum/.git/hoo